In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve, least_squares
from tqdm.notebook import tqdm as tqdm

In [2]:
def plt_spectrum_header():
    plt.figure(dpi=100, figsize=(8,4.5))
    plt.minorticks_on()
    plt.grid(True, which='both', axis='x')
    plt.grid(True, which='major', axis='y')
    plt.xlim([400, 5000])
    plt.xlabel('$\lambda$, cm$^{-1}$')

In [3]:
%run Data.ipynb

In [4]:
def index(L):
    return int(Lambda.shape[0] * (L - Lambda[0])/(Lambda[-1] - Lambda[0]))

In [14]:
### for testing ###
part = False

if part is True:
    l_min = 3000
    l_max = 3150
    n = n_PEG
    f0 = NaPB[index(l_max):index(l_min)]
    f1 = PEG_Pure[index(l_max):index(l_min)]
    f = PEG[:, index(l_max):index(l_min)]
else:
    n = n_PEG
    f0 = NaPB
    f1 = PEG_Pure
    f = PEG

# Two-Component Solutions

In [5]:
def Find_2Concentrations(F, f0, f1):
    
    Results = [
        least_squares(fun=lambda x, f, f0, f1 : (x[0]*f0 + x[1]*f1 - f),
                      x0=np.zeros(2), 
                      args=(f, f0, f1)).x
        for f in F
    ]
        
    Results = np.asarray(Results)
    
    return Results

In [ ]:
def Find_3Concentrations(F, f0, f1, f2):
    
    Results = [
        least_squares(fun=lambda x, f, f0, f1, f2 : (x[0]*f0 + x[1]*f1 + x[2]*f2 - f).flatten(),
                      x0=np.zeros(3),
                      args=(f, f0, f1, f2)).x
        for f in F
    ]
        
    Results = np.asarray(Results)
    
    return Results

In [ ]:
def Finde_Pure(F, n, F0):
    
    Results = [
        least_squares(fun=lambda x, f, n, f0 : (n*x + (1 - n)*f0 - f).flatten(), 
                      x0=0,
                      args=(f, n, f0)).x
        for f, f0 in zip(F.T, F0):
    ]
        
    Results = np.asarray(Results).squeeze()
    
    return Results

### Model
We assume that there are complexes of solvent and solute.

$f = c_0 f_0 + c_1 f_1 + c_{01} f_{01}$

$c_{01} = k {c_0^{\alpha_0} c_1^{\alpha_1}}$

$n_0 = c_0 + \alpha_0 c_{01}$

$n_1 = c_1 + \alpha_1 c_{01}$

### Variabels

$f$ is the absorbence fo the solution, 

$f_0$, $f_1$ and $f_{01}$ are there absorbencees of the components (solvent, solute and complex),

$c_0$, $c_1$ and $c_{01}$ are the concentrations,

$k$ is the raction coefficient

$\alpha_0$ and $\alpha_1$ are the numbers of atoms in the complex

$n_0$ and $n_1$ are the initia concentrations

In [6]:
def Concentrations(k, a0, a1, n):
    """
    finds concentrations of both pure components, and the complex
    
    k, a0, a1 : scaliars or arrays of shape (M)
    n : array of shape (N)
    
    returns : 3 arrays of shape (N, M)
    """
    k, a0, a1, n = np.atleast_1d(k, a0, a1, n)
    
    c01 = np.empty((n.shape[0], k.shape[0]))
    
    for i in range(n.shape[0]):
        for j in range(k.shape[0]):
            
            c01[i,j] = fsolve(Fun_For_Concentrations, 0,
                              (k[j], a0[j], a1[j], n[i]))[0]
    
    c0 = 1 - n[:, None] - a0[None, :]*c01
    c1 = n[:, None] - a1[None, :]*c01
    
    return c0, c1, c01
 
    
def Fun_For_Concentrations(c01, k, a0, a1, n):
    """
    solving f(c01)=0 we find c01
    """
    c0 = 1 - n - a0*c01
    c1 = n - a1*c01
    c01_model = k * (np.power(c0, a0)) * (np.power(c1, a1))
    return c01_model - c01


def Fun_For_Concentrations_Prime(c01, k, a0, a1, n):
    """
    derivative of f(c01)
    """
    c0 = 1 - n - a0*c01
    c1 = n - a1*c01
    
    with np.errstate(divide='ignore', invalid='ignore'):
        common = k * np.power(c0, a0-1) * np.power(c1, a1-1)
        fprime = - 1 - common * (a0**2 * c1 + a1**2 * c0)
        
    return fprime


In [8]:
def Predict(c0, c1, c01, f0, f1, f01):
    """
    c0, c1, c01 : arrays of shape (N, M) or (N, 1)
    f0, f1, f01 : arrays of shape (M)
    
    returns : array of shape (M) or (N, M)
    """
    f0, f1, f01 = np.atleast_1d(f0, f1, f01)
    
    if c01.shape[1] == 1:
        f = c0[:,0,None]*f0[None, :]
        f += c1[:,0,None]*f1[None, :]
        f += c01[:,0,None]*f01[None, :]
    else:
        f = c0*f0[None, :]
        f += c1*f1[None, :]
        f += c01*f01[None, :]
    
    return f


# Not Including lstsq

In [43]:
def fun_mI(x, f, n):
    k, a0, a1, f0, f1, f01 = x
    
    c0, c1, c01 = Concentrations(k, a0, a1, n)
    f_pred = Predict(c0, c1, c01, f0, f1, f01)
    
    return (f_pred - f).flatten()


def common_v_prime(k, a0, a1, c0, c1, c01):
    
    with np.errstate(divide='ignore', invalid='ignore'):
        v = np.array([1/k * np.ones(c01.shape),
                      np.log(c0) - a0*c01/c0,
                      np.log(c1) - a1*c01/c1])
        v /= (1/c01 + a0**2/c0 + a1**2/c1)
    
    v[:, c0==0] = 0
    v[:, c1==0] = 0
    v[:, c01==0] = 0
    
    return v
    
def fun_prime_mI(x, f, n):
    k, a0, a1, f0, f1, f01 = x
    
    c0, c1, c01 = Concentrations(k, a0, a1, n)
    v = common_v_prime(k, a0, a1, c0, c1, c01)
    
    f_prime = np.array([0, - f0*c01, - f1*c01, c0, c1, c01])
    f_prime[0:3] -= v*(f01-a0*f0-a1*f1)
    
    return f_prime

In [ ]:
def Estimate_mI(f, n, batch=10):
    """
    f : array of shape (N, M)
    n : array of shape (N)
    batch : int
    
    returns : array of resulrts that includes (k, a0, a1)
              arrays of shape (M + 1 - batch)
    """
    
    Results = []
    for i in tqdm(range(0, len(f0) + 1 - batch), int(batch/2)):
        x0 = .5*np.ones(6)
        res = least_squares(fun_mI, x0,
                            args=(f[:, i:i+batch], n),
                            bounds=(0, [np.inf, np.inf, np.inf, 1, 1, 1]))
        
        
        Results.append(res)
    
    return Results

# Including lstsq

In [ ]:
def f0_f1_f01(c0, c1, c01, f, negative=False):
    """
    c0, c1, c01 : arrays of shape (N, 1) or (N, M)
    f : array of shape (N) or (N, M)
    
    returns : array of shape (3) or (3, M)
    """
    
    A = np.array([c0, c1, c01]).T.squeeze()
    B = f.copy().T

    Solution = []
    for i, b in enumerate(B):
        
        if c01.shape[1] == 1:
            a = A
        else:
            a = A[i]
        
        sol = np.linalg.lstsq(a, b, rcond=None)
        
        Gt0 = sol[0] > 0
        Lt1 = sol[0] < 1
        
        if all(Gt0 & Lt1) or negative:
            Solution.append(sol[0])
            
        else:
            b -= a[:, np.logical_not(Lt1)].sum(axis=1)
            a = a[:, (Gt0 & Lt1)]
            sol = np.linalg.lstsq(a, b, rcond=None)
        
            s = []
            j = 0
            for gt0, lt1 in zip(Gt0, Lt1):
                if gt0 & lt1:
                    s.append(sol[0][j])
                    j += 1
                elif not gt0:
                    s.append(0)
                elif not lt1:
                    s.append(1)
            Solution.append(s)
            
    Solution = np.asarray(Solution).T
    
    return Solution

In [ ]:
def fun_I(x, f, n):
    
    k, a0, a1 = x
    
    if a0 < 0:
        a0 = 0
    if a1 < 0:
        a1 = 0
    
    c0, c1, c01 = Concentrations(k, a0, a1, n)
    f0, f1, f01 = f0_f1_f01(c0, c1, c01, f)
    f_pred = Predict(c0, c1, c01, f0, f1, f01)
    
    return (f_pred - f).flatten()

In [ ]:
def Estimate_I(f, n, batch=10, reduced=True):
    """
    f : array of shape (N, M)
    n : array of shape (N)
    batch : int
    
    returns : array of resulrts that includes (k, a0, a1)
              arrays of shape (M + 1 - batch)
    """
    
    Results = []
    length = len(f[0])+1-batch
    step = int(batch/2)
    
    for i in tqdm(range(0, length, step) if reduced is True else range(length)):
        x0 = np.ones(3)
        res = least_squares(fun_I, x0,
                            args=(f[:, i:i+batch], n),
                            bounds=(0, [1, np.inf, np.inf]))
        
        Results.append(res)
    
    return Results

## Liquid

$f_{01} = \frac{1}{\sum c_{01}^2} \left[\sum f c_{01} - f_0 \sum c_{01} c_0 - f_1 \sum c_{01} c_1 \right]$

In [9]:
def Liquid_f01(c0, c1, c01, f0, f1, f):
    """
    c0, c1, c01 : arrays of shape (N, 1) or (N, M)
    f0, f1 : scalars or arrays of shape (M)
    f : array of shape (N) or (N, M)
    
    returns : scalar or array of shape (M)
    """
    f0, f1 = np.atleast_1d(f0, f1)
    if f.ndim == 1:
        f = f[:,None]
    
    if (c01**2).sum() == 0:
        f01 = np.zeros(f0.shape)
    
    else:
        if c01.shape[1] == 1:
            f01 = (f * c01[:,0,None]).sum(axis=0)
            f01 -= f0 * (c0 * c01).sum(axis=0)
            f01 -= f1 * (c1 * c01).sum(axis=0)
            f01 /= (c01**2).sum(axis=0)
        else:
            f01 = (f * c01).sum(axis=0)
            f01 -= f0 * (c0 * c01).sum(axis=0)
            f01 -= f1 * (c1 * c01).sum(axis=0)
            f01 /= (c01**2).sum(axis=0)
            
        f01[f01 < 0] = 0 
        f01[f01 > 1] = 1
    
    return f01


In [11]:
def Liquid_fun_I(x, n, f0, f1, f):
    """
    x : array of shape (3)
    n : array of shape (N)
    f0, f1 : arrays of shape (batch)
    f : array of shape (N, batch)
    
    return : array of shape (N * batch)
    loss function to mminimize
    """
    k, a0, a1 = x[0], x[1], x[2]
    
    if a0 < 0:
        a0 = 0
    if a1 < 0:
        a1 = 0
    
    c0, c1, c01 = Concentrations(k, a0, a1, n)
    f01 = Liquid_f01(c0, c1, c01, f0, f1, f)
    f_pred = Predict(c0, c1, c01, f0, f1, f01)
    
    return (f_pred - f).flatten()


In [12]:
def Liquid_fun_II(x, a0, a1, n, f0, f1, f):
    """
    x : array of shape (3)
    n : array of shape (N)
    f0, f1 : arrays of shape (batch)
    f : array of shape (N, batch)
    
    return : array of shape (N * batch)
    loss function to minimiza
    """
    k = x
    
    if a0 < 0:
        a0 = 0
    if a1 < 0:
        a1 = 0
    
    c0, c1, c01 = Concentrations(k, a0, a1, n)
    f01 = Liquid_f01(c0, c1, c01, f0, f1, f)
    f_pred = Predict(c0, c1, c01, f0, f1, f01)
    
    return (f_pred - f).flatten()

In [13]:
def Estimate_Liquid_k_a0_a1_I(n, f, batch):
    """
    n : array of shape (N)
    f0, f1 : arrays of shape (M)
    f : array of shape (N, M)
    batch : int
    
    returns : array of resulrts that includes (k, a0, a1)
              arrays of shape (M + 1 - batch)
    """    
    
    f0 = f[0]
    f1 = f[-1]
    f = f[1:-1]
    n = n[1:-1]
    
    Results = []
    for i in tqdm(range(len(f0) + 1 - batch)):
        x0 = np.ones(3)
        res = least_squares(Liquid_fun_I, x0,
                            args=(n, f0[i:i+batch], f1[i:i+batch], f[:, i:i+batch]),
                            bounds=(0, np.inf))
        
        Results.append(res)
    
    return Results


In [ ]:
def get_k_a0_a1_I(Results):
    k = []
    a0 = []
    a1 = []

    for i in range(len(Results)):
        k.append(Results[i].x[0])
        a0.append(Results[i].x[1])
        a1.append(Results[i].x[2])
    
    k = np.array(k)
    a0 = np.array(a0)
    a1 = np.array(a1)
    
    return k, a0, a1

In [14]:
def Estimate_Liquid_k_a0_a1_II(a0, a1, n, f0, f1, f, History, batch):
    
    a0 = a0.astype(int)
    a1 = a1.astype(int)
    
    add = [[0,0],[1,0],[0,1],[1,1]]
    
    Results = []
    for i in tqdm(range(len(f0) + 1 - batch)):
        reses = []
        losses = []
        for j in add:
            res = least_squares(Liquid_fun_II, x0=0,
                                args=(a0[i]+j[0], a1[i]+j[1], n,
                                      f0[i:i+batch], f1[i:i+batch],
                                      f[:, i:i+batch]),
                                bounds=(0, np.inf))
            reses.append(res)
            losses.append(res.cost)
            
        index = losses.index(min(losses))
        Results.append((reses[index],
                        a0[i]+add[index][0],
                        a1[i]+add[index][1]))
        
    Estimation = [Results, batch]
    History.append(Estimation)
    
    return Results


In [ ]:
def get_k_a0_a1_II(Results):
    k = []
    a0 = []
    a1 = []

    for i in range(len(Results)):
        k.append(Results[i][0].x)
        a0.append(Results[i][1])
        a1.append(Results[i][2])
    
    k = np.array(k)
    a0 = np.array(a0)
    a1 = np.array(a1)
    
    return k, a0, a1

## Solid

$f_1 \sum c_1 c_{01} + f_{01} \sum c_{01}^2   = \sum f c_{01} - f_0 \sum c_0 c_{01}$

$f_1 \sum c_1^2      + f_{01} \sum c_{01} c_1 = \sum f c_1    - f_0 \sum c_0 c_1$

In [15]:
def Solid_f1_f01(c0, c1, c01, f0, f):
    """
    c0, c1, c01 : arrays of shape (N, 1) or (N, M)
    f0, f1 : scalars or arrays of shape (M)
    f : array of shape (N) or (N, M)
    
    returns : scalars or arrays of shape (M)
    """
    f0 = np.atleast_1d(f0)
    if f.ndim == 1:
        f = np.atleast_2d(f).T
        
    if c01.shape[1] == 1:
        
        a = np.array([[(c1 * c01).sum(), (c01**2).sum()  ],
                      [(c1**2).sum(),    (c1 * c01).sum()]])
        b = np.array([(f * c01[:,0,None]).sum(axis=0)-f0 * (c0 * c01).sum(),
                      (f * c1[:,0,None]).sum(axis=0) -f0 * (c0 * c1).sum() ]).T
        
        f1, f01 = np.linalg.solve(a[None, ...], b)
            
    else:
        
        a = np.array([[(c1 * c01).sum(axis=0), (c01**2).sum(axis=0)  ],
                      [(c1**2).sum(axis=0),    (c1 * c01).sum(axis=0)]])
        a = np.moveaxis(a, -1, 0)
        b = np.array([(f * c01).sum(axis=0) - f0 * (c0 * c01).sum(axis=0),
                      (f * c1).sum(axis=0)  - f0 * (c0 * c1).sum(axis=0) ]).T
        
        f1, f01 = np.linalg.solve(a, b)
            
    f1[f1 < 0] = 0
    f1[f1 > 1] = 1
    f01[f01 < 0] = 0 
    f01[f01 > 1] = 1
        
    return f1, f01


In [ ]:
def Solibatchoss(k, a0, a1, n, f0, f): 
    
    c0, c1, c01 = Concentrations(k, a0, a1, n)
    f1, f01 = Solid_f1_f01(c0, c1, c01, f0, f)
    f_pred = Predict(c0, c1, c01, f0, f1, f01)
    return ((f_pred - f)**2).sum()


In [16]:
def Solid_fun_I(x, n, f0, f):
    """
    x : array of shape (3)
    n : array of shape (N)
    f0 : array of shape (batch)
    f : array of shape (N, batch)
    
    return : array of shape (N * batch)
    """
    k, a0, a1 = x[0], x[1], x[2]
    
    if a0 < 0:
        a0 = 0
    if a1 < 0:
        a1 = 0
    
    c0, c1, c01 = Concentrations(k, a0, a1, n)
    f1, f01 = Solid_f1_f01(c0, c1, c01, f0, f)
    f_pred = Predict(c0, c1, c01, f0, f1, f01)
    
    return (f_pred - f).flatten()


In [17]:
def Solid_fun_II(x, a0, a1, n, f0, f):
    """
    x : array of shape (3)
    n : array of shape (N)
    f0 : arrays of shape (batch)
    f : array of shape (N, batch)
    
    return : array of shape (N * batch)
    """
    k = x
    
    if a0 < 0:
        a0 = 0
    if a1 < 0:
        a1 = 0
    
    c0, c1, c01 = Concentrations(k, a0, a1, n)
    f1, f01 = Solid_f1_f01(c0, c1, c01, f0, f)
    f_pred = Predict(c0, c1, c01, f0, f1, f01)
    
    return (f_pred - f).flatten()


In [18]:
def Estimate_Solid_k_a0_a1_I(n, f0, f, Estimation_History, batch):
    """
    n : array of shape (N)
    f0 : array of shape (M)
    f : array of shape (N, M)
    batch : int
    
    returns : array of resulrts that includes (k, a0, a1)
              arrays of shape (M + 1 - batch)
    """    
    Results = []
    for i in tqdm(range(len(f0) + 1 - batch)):
        x0 = np.ones(3)
        res = least_squares(Solid_fun_I, x0,
                            args=(n, f0[i:i+batch], f[:, i:i+batch]),
                            bounds=(0, np.inf))
        
        Results.append(res)
    
    
    Estimation = [Results, batch]
    Estimation_History.append(Estimation)
    
    return Results


In [ ]:
def Estimate_Solid_k_a0_a1_II(a0, a1, n, f0, f, Estimation_History, batch):
    
    a0 = a0.astype(int)
    a1 = a1.astype(int)
    
    add = [[0,0],[1,0],[0,1],[1,1]]
    
    Results = []
    for i in tqdm(range(len(f0) + 1 - batch)):
        reses = []
        losses = []
        for j in add:
            res = least_squares(Solid_fun_II, x0=0,
                                args=(a0+j[0], a1+j[1], n,
                                      f0[i:i+batch], f1[i:i+batch],
                                      f[:, i:i+batch]),
                                bounds=(0, np.inf))
            reses.append(res)
            losses.append(res.cost)
            
        index = losses.index(min(losses))
        Results.append(reses[index])
        
    Estimation = [Results, batch]
    History.append(Estimation)
    
    return Results
